# Яндекс Музыка

Приложением Яндекс Музыка предоставлены данные о музыке, которую слушают пользователи из Москвы и Санкт-Петербурга. Есть несколько мифов о городах, например, что Москва - мегаполис, подчинённый жёсткому ритму рабочей недели, а Питер - культурная столица со своеобразными вкусами. На данных Яндекс Музыки необходимо сравнить поведение пользователей двух столиц. Целью исследования является проверка трёх гипотез:
- Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
- В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
- Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

<font size=+1><b>Данные</b></font><br>
Данные с пользователями music_project.csv
- `userID` — уникальный ID пользователя
- `Track` - название музыкального трека
- `artist` - исполнитель музыкального трека
- `genre` - жанр музыкального трека
- `City` - город пользователя
- `time` - время прослушивания музыкального трека
- `Day` - день, в который пользователь слушал музыкальный трек

<font size=+1><b>План работы:</b></font><br>
- Загрузить и изучить данные
- Найти и изучить пропуски в данных
- Найти и обработать дубликаты в данных
- Проверить гипотезу о том, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге по дням недели
- Проверить гипотезу о том, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города
- Проверить гипотезу о том, Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве. А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка
- Сделать выводы по проведённому исследованию

## Загрузка данных

### Используемые библиотеки и функции

In [1]:
import pandas as pd

In [2]:
# Цветовая палитра графиков
cmap_table='coolwarm'

In [3]:
# функция для загрузки данных
# функция принимает как параметр строку с путём к файлу
# функция возвращает датафрейм и информацию по нему
def main_info (dataset):
    data = pd.read_csv(dataset)
    display(data.head())
    print('------------------------------------------')
    display('Основная информация по столбцам таблицы:')
    display(data.info())
    print('-------------------------------------')
    display('Доля пропусков по столбцам таблицы:')
    display(pd.DataFrame(round(data.isna().mean()\
                                   .sort_values(ascending=False), 3))\
                        .style.format('{:.1%}')\
                        .background_gradient(cmap_table))
    print('----------------------------------------')
    display('Количество явных дубликатов в таблице:')
    display(data.duplicated().sum())
    return data

In [4]:
# функция для поиска дубликатов в названии жанра
def find_genre(genre):
    k = 0
    for genre_cicle in genres_list:
        if genre_cicle == genre:
            k += 1
    return k

In [5]:
# функция для создания сводной таблицы
def table_pivot(data, col_group, values, func):
    return data.groupby(by=col_group, as_index=False)\
               .agg(total = (values, func))

In [6]:
# функция для выявления топ жанров
def genre_weekday(df, day, time1, time2):
    genre_df = table_pivot(data=df.query('day == @day & time <= @time2 & time >= @time1'), 
                           col_group='genre', values='genre', func='count')
    return genre_df.sort_values(by='total', ascending = False).head(10).reset_index(drop=True)

### Загрузка данных и основная информация

Посмотрим на основную информацию из таблицы:

In [7]:
music = main_info('music_project.csv')

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


------------------------------------------


'Основная информация по столбцам таблицы:'

<class 'pandas.core.frame.DataFrame'>
Index: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userID    65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 4.0+ MB


None

...

'Доля пропусков по столбцам таблицы:'

,0
artist,11.1%
Track,1.9%
genre,1.8%
userID,0.0%
City,0.0%
time,0.0%
Day,0.0%


----------------------------------------


'Количество явных дубликатов в таблице:'

3826

### Промежуточные выводы:

- В таблице 7 столбцов и 65079 записей с информацией о треках, которые слушают пользователи приложения.
- В трёх столбцах есть пропуски значений.
- Доля пропусков в столбце с исполнителем занимет 11.1% от общего количества строк в датафрейме.
- В остальных столбцах с названием трека и жанре композиции пропуски занимают до 2% данных.
- В таблице есть 3826 явных дубликатов.
- Названия столбцов не соответствуют PEP-8.

## Предобработка данных

### Обработка пропусков

In [8]:
# Переменная для контроля % удалённых значений
del_value = [0, music.shape[0]]

Посмотрим на названия столбцов.

In [9]:
music.columns

Index(['userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

В названиях некоторых столбцов присутствуют лишние пробелы, также некоторые названия нужно привести к нижнему регистру. Заменим название столбцов согласно PEP-8.

In [10]:
music.columns = music.columns.str.lower()
music = music.rename(columns={'userid': 'user_id', '  city  ': 'city'})

В таблице в трёх столбцах были пропуски данных: название произведения, исполнитель трека и жанр. Пропуски значений означают, что для некоторых музыкальных треков доступна не вся информация: не указан исполнитель, нет определённого жанра. Так как для анализа столбец с жанром музыкального произведения важен, то все строки с пропущенным жанром можно удалить. В остальных столбцах пропуски можно заменить на "неизвестно", так как восстановить эту информацию не предоставляется возможным.

In [11]:
music.dropna(subset = ['genre'], inplace = True)
del_value[0] = music.shape[0]

In [12]:
columns_to_replace = ['track', 'artist']
for df_replace in columns_to_replace:
    music[df_replace] = music[df_replace].fillna('unknown')

In [13]:
print('Процент удалённых значений: ', 
      "{0:.1%}".format((del_value[1] - del_value[0]) / del_value[1]))

Процент удалённых значений:  1.8%


Проверим, что все пропуски были исправлены на значения.

In [14]:
music.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Удаление дубликатов

В таблице было 3826 строчек явных дубликатов. Дубликаты могли появиться вследствие сбоя в записи данных. Удалим их.

In [15]:
music = music.drop_duplicates().reset_index(drop=True)
del_value[0] = music.shape[0]

Проверим неявные дубликаты, которые могли оказаться в колонке с жанром музыкального трека. Сохраним список уникальных значений столбца с жанрами в переменной genres_list.

In [16]:
genres_list = music['genre'].unique()
print('Количество уникальных жанров: ', music['genre'].nunique())

Количество уникальных жанров:  287


В таблице могли оказаться разные варианты названия жанра хип-хоп. Вызовем функцию find_genre() для поиска различных вариантов названия жанра хип-хоп в таблице.

Правильное название — hiphop. Поищем другие варианты:
- hip
- hop
- hip-hop

In [17]:
print('Количество правильного написания жанра hiphop: ', find_genre('hiphop'))
print('Количество написания жанра hip: ', find_genre('hip'))
print('Количество написания жанра hop: ', find_genre('hop'))
print('Количество написания жанра hip-hop: ', find_genre('hip-hop'))

Количество правильного написания жанра hiphop:  1
Количество написания жанра hip:  1
Количество написания жанра hop:  0
Количество написания жанра hip-hop:  0


Как видно, для названий жанра hiphop есть всего один неявный дубликат: *hip*. Приведём его к принятому обозначению.

In [18]:
music['genre'] = music['genre'].replace('hip', 'hiphop')
print('Количество уникальных жанров: ', music['genre'].nunique())

Количество уникальных жанров:  286


Проверим, что из таблицы были удалены все явные дубликаты.

In [19]:
music.duplicated().sum()

0

Получим общую информацию о данных. Убедимся, что предобработка выполнена успешно.

In [20]:
music.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  60126 non-null  object
 1   track    60126 non-null  object
 2   artist   60126 non-null  object
 3   genre    60126 non-null  object
 4   city     60126 non-null  object
 5   time     60126 non-null  object
 6   day      60126 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


In [21]:
print('Процент удалённых значений: ', 
      "{0:.1%}".format((del_value[1] - del_value[0]) / del_value[1]))

Процент удалённых значений:  7.6%


### Промежуточные выводы:

Была проведена предобработка данных для таблицы с музыкальными треками пользователей.
- Заголовки столбцов были приведены к принятому стилю.
- Доля пропусков в столбце с исполнителем занимала 11.1% от общего количества строк в датафрейме.
- В остальных столбцах с названием трека и жанре композиции пропуски занимали до 2% данных.
- Строчки, в которых были пропуски данных о жанре трека, были удалены.
- Пропуски в остальных столбцах были заменены на значение "неизвестно".
- В таблице присутствовало 3826 записей явных дубликатов, они были удалены из таблицы.
- Прроцент удалённых значений составил 7.6% от данных датафрейма.
- В столбце с жанром трека присутствовали разные варианты записи жанра hiphop, они были приведены к правильному названию.
- После замены осталось 286 уникальных жанра вместо 287.
- Данные подготовлены для дальнейшего анализа.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Необходимо проверить это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Оценим активность пользователей в каждом городе. 

In [22]:
table_pivot(data=music, col_group='city', values='user_id', func='count')

,city,total
0,Moscow,41892
1,Saint-Petersburg,18234


В Москве прослушиваний больше, чем в Петербурге, но и самых пользователей в Москве также больше, чем в Питере. Так что из этого не следует, что московские пользователи чаще слушают музыку.

Подсчитаем прослушивания в понедельник, среду и пятницу для двух городов одновременно.

In [23]:
table_pivot(data=music, col_group='day', values='user_id', func='count')

,day,total
0,Friday,21482
1,Monday,20866
2,Wednesday,17778


В среднем пользователи из двух городов менее активны по средам и более вовлечены в прослушивание музыки по пятницам и понедельникам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [24]:
pd.pivot_table(music, values='user_id', index=['city'], columns=['day'], aggfunc='count')

day,Friday,Monday,Wednesday
city,,,
Moscow,15680,15347,10865
Saint-Petersburg,5802,5519,6913


Данные показывают разницу поведения пользователей:
- В Москве пик прослушиваний приходится на понедельник и пятницу? причём в почти равной мере, а в среду заметен сильный спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы о том, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [25]:
# Выделим отдельно пользователей из Москвы и Питера
moscow_general = music.query('city == "Moscow"')
spb_general = music.query('city == "Saint-Petersburg"')

Cравним результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11).

In [26]:
# Москва в понедельник утром
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

,genre,total
0,pop,781
1,dance,549
2,electronic,480
3,rock,474
4,hiphop,286
5,ruspop,186
6,world,181
7,rusrap,175
8,alternative,164
9,classical,157


In [27]:
# Питер в понедельник утром
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

,genre,total
0,pop,218
1,dance,182
2,rock,162
3,electronic,147
4,hiphop,80
5,ruspop,64
6,alternative,58
7,rusrap,55
8,jazz,44
9,classical,40


Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. В понедельник утром в топе находится энергичная музыка жанра "pop" и "dance". 
2. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — “джаз“.

In [28]:
# Москва в пятницу вечером
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

,genre,total
0,pop,713
1,rock,517
2,dance,495
3,electronic,482
4,hiphop,273
5,world,208
6,ruspop,170
7,alternative,163
8,classical,163
9,rusrap,142


In [29]:
# Питер в пятницу вечером
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

,genre,total
0,pop,256
1,rock,216
2,electronic,216
3,dance,210
4,hiphop,97
5,alternative,63
6,jazz,61
7,classical,60
8,rusrap,59
9,world,54


Вечер пятницы не сильно меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым. Единственное, что в пятницу в Питере в топе появляется жанр "world" и пропадает жанр "ruspop".

Таким образом, вторая гипотеза подтвердилась лишь частично:
- Пользователи слушают похожую музыку в начале недели и в конце.
- Разница между Москвой и Петербургом не слишком выражена.
- Жанр поп безусловный лидер и в понедельник утрои и в пятницу вечером, а топ-5 в целом не различается в обеих столицах.
- Конец списка немного более выражен для обеих столиц в разные дни недели и разное время суток.
- В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве. А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [30]:
# топ-10 прослушиваемые жанры Москвы
table_pivot(data=moscow_general, col_group='genre', values='genre', func='count')\
           .sort_values(by='total', ascending = False).head(10).reset_index(drop=True)

,genre,total
0,pop,5892
1,dance,4435
2,rock,3965
3,electronic,3786
4,hiphop,2096
5,classical,1616
6,world,1432
7,alternative,1379
8,ruspop,1372
9,rusrap,1161


In [31]:
# топ-10 прослушиваемые жанры Питера
table_pivot(data=spb_general, col_group='genre', values='genre', func='count')\
           .sort_values(by='total', ascending = False).head(10).reset_index(drop=True)

,genre,total
0,pop,2431
1,dance,1932
2,rock,1879
3,electronic,1736
4,hiphop,960
5,alternative,649
6,classical,646
7,rusrap,564
8,ruspop,538
9,world,515


Гипотеза частично подтвердилась:
- Поп-музыка — самый популярный жанр в Москве, что частично подтверждает гипотезу, но также он самый популярный жанр и в Питере.
- Топ-5 жанров одинаков для Москвы и для Питера.
- В топ-10 жанров для обоих городов встречается близкий жанр — русская популярная музыка.
- Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 

## Итоги исследования

Было проведено исследование по теме музыкальных предпочтений жителей двух городов - Питера и Москвы.

Перед анализом, данные были подготовлены для корректной работы:
- В таблице содержалось 7 столбцов и 65079 записей с информацией о треках, которые слушают пользователи приложения.
- В трёх столбцах были пропуски значений, причём в одном столбце доля пропусков составляла 11.1%.
- Строчки с пропуском данных в жанре трека были удалены.
- В остальных столбцах пропуски были заменены на значение "неизвестно".
- В таблице были 3826 записей явных дубликатов, они были удалены из таблицы.
- В столбце с жанром трека присутствовали разные варианты записи жанра hiphop, они были приведены к правильному названию.
- Процент удалённых значений составил 7.6% от данных датафрейма.

Были проверены три гипотезы:
1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.
2. Музыкальные предпочтения меняются в течение недели.
3. В Москве и Петербурге разные музыкальные вкусы: в Петербурге преобладает рэп, а в Москве — поп-музыка.

Первая гипотеза полностью подтвердилась. Данные показывают разницу поведения пользователей:
- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Вторая гипотеза подтвердилась лишь отчасти.
- Музыкальные предпочтения не сильно меняются в течение недели и для Москвы, и для Питера.
- Топ-5 предпочтений в целом не различается в обеих столицах.
- Конец списка немного более выражен для обеих стлиц в разные дни недели и разное время суток.
- В Москве чаще слушают русскую популярную музыку, а в Петербурге — джаз.

Третья гипотеза не подтвердилась. Самая популярная музыка в обоих городах - поп. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Таким образом, можно подвести общий итог: музыкальные предпочтения пользователей из Москвы и Санкт-Петербурга мало отличаются друг от друга, пользователи слушают музыку схожим образом, но вот активны в разных городах в разные дни.